## Assignments : Python programming
---
Author : Eun-Seok Ji

In [1]:
import time
import traceback
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

### Crawing Process
---
* Chrome browser 기준으로 시크릿 옵션을 주어 실행합니다.

1. 개발 탭을 찾아서 이동한다.

2. 파이썬 개발자 탭으로 이동한다.
* 파이썬 개발자 탭을 선택하기 위해서 탭을 넘겨주는 동작 필요합니다.
    * Web page 분석 결과 tab elements는 이미 생성되어 있지만 Tab 내부 글자는 탭 이동 버튼(arrowBtn) 클릭을 수행해야 생성되는 모습을 보입니다.
    * inner text가 없을 경우 버튼을 클릭합니다.
    * 현재 인덱스를 다시 읽는 동작을 수행합니다.

3. 먼저 스크롤을 수행합니다. 
* 분석 결과, Scrollin 수행 시 동적으로 'Job-Card' li element가 생성됩니다.
    * 직관적인 코드 구조를 위해 먼저 스크롤을 수행
    * 개수 확인 후 100개가 생성되면 읽어오는 과정 수행

4. 생성된 정보를 Crawling하여 수집합니다.


In [4]:
path_webdriver = '/Users/eunseokji/Workspaces/LecPythonWantedCrawling/'
WebDrv = 'chromedriver'
url = 'https://www.wanted.co.kr/wdlist?country=kr&job_sort=job.latest_order&years=-1'

wantedList = []
idx = 0

Options = webdriver.ChromeOptions()
# Secret window option
Options.add_argument('--incognito')
drv = webdriver.Chrome(path_webdriver + WebDrv, options=Options)

try:
    drv.get(url)
    time.sleep(3)

    # print('Start parsing')
    # Process 1
    Tabs = drv.find_elements_by_class_name('_3SgvgiuDypnw8sSW2Pxngs')
    for tab in Tabs:
        # print(tab.text)
        DevTab = tab.find_element_by_tag_name('a')
        TabLink = DevTab.get_attribute('href')
        
        if tab.text.strip() == '개발':
            TabLink = DevTab.get_attribute('href')
            break
    
    # Move page
    drv.get(TabLink)
    drv.implicitly_wait(10)
    
    # Process 2
    Tabs = drv.find_elements_by_class_name('_3SgvgiuDypnw8sSW2Pxngs')
    
    while idx < len(Tabs):
        # print(Tabs[idx].text)
        # Terminated condition
        if Tabs[idx].text.strip() == '파이썬 개발자':
            # Move page
            Tabs[idx].click()
            break
        
        # Next button click
        elif Tabs[idx].text.strip() == "":
            arrowBtn = drv.find_element_by_class_name('next')
            arrowBtn.click()
            idx = idx - 1
        
        idx = idx + 1
    
    #drv.implicitly_wait(10)
    time.sleep(2)
    
    # Process 3
    JobCards = drv.find_elements_by_css_selector('.clearfix > li')
    while len(JobCards) < 100:
        drv.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)
        JobCards = drv.find_elements_by_css_selector('.clearfix > li')
    
    # Process 4
    JobCards = drv.find_elements_by_css_selector('.clearfix > li')
    for card in JobCards:
        position = card.find_element_by_class_name('job-card-position').text
        company = card.find_element_by_class_name('job-card-company-name').text
        location = card.find_element_by_class_name('job-card-company-location').text.split('.')[0]
        reward = card.find_element_by_class_name('reward').text.split(' ')[1][:-1]
        #print(position, company, location, reward)
        like = card.find_element_by_class_name('likeButton').text
        #print(like)
        #print('='*100)
        tmpList = [position, company, location, reward, like]
        wantedList.append(tmpList)
        
except:
    traceback.print_exc()
finally:
    time.sleep(3)
    #print('Finally?')
    drv.quit()

    
#print(wantedList)
print(len(wantedList))

100


### Data Analysis
---
* 먼저 DateFrame을 생성하고 수치계산이 가능하도록 Type casting 합니다.

In [5]:
def make_int(reward):
    reward = reward.replace(',', '')
    reward = int(reward)
    return reward

df = pd.DataFrame(wantedList)
df.columns = ['Position', 'Company', 'Location', 'Reward', 'Like']
df = df.astype({'Position':'string', 'Company':'string', 'Location':'string', 'Reward':'string', 'Like':'int'})

df.Reward = df.Reward.apply(make_int)

df

,Position,Company,Location,Reward,Like
0,iOS 앱 개발자,모딧,서울,1000000,25
1,Flutter 모바일 앱 개발자,소프트런치,서울,1000000,14
2,QA 엔지니어,"펀엔씨(강아지대통령,고양이대통령)",서울,1000000,19
3,Backend Engineer,오픈메이트온,서울,1000000,2
4,테스트 엔지니어 (리더급),딜리셔스(신상마켓),서울,1000000,2
...,...,...,...,...,...
95,"백엔드 개발자(JAVA,SPRING)",클래스101(Class101),서울,1000000,246
96,신사업 백엔드 개발자(Go),코빗(Korbit),서울,1000000,0
97,솔루션 아키텍트(Solutions Architect),에이비일팔공(AB180),서울,1000000,54
98,프론트엔드 엔지니어,에멘탈(비즈넵),서울,1000000,7


* 수집한 전체 데이터의 채용 보상금, 좋아요의 통계적 정보를 확인합니다.

In [6]:
df.describe()

,Reward,Like
count,100.0,100.000000
mean,1000000.0,20.740000
std,0.0,38.018767
min,1000000.0,0.000000
25%,1000000.0,1.000000
50%,1000000.0,8.000000
75%,1000000.0,21.250000
max,1000000.0,246.000000


* 채용공고의 지역별 '좋아요' 수를 확인합니다.

In [7]:
df.Location.value_counts()

서울    95
경기     4
대전     1
Name: Location, dtype: Int64

In [8]:
df_sub = df.iloc[:,1:]
df_sub.groupby(['Location']).describe()

Reward                                                              \
          count       mean  std        min        25%        50%        75%   
Location                                                                      
경기          4.0  1000000.0  0.0  1000000.0  1000000.0  1000000.0  1000000.0   
대전          1.0  1000000.0  NaN  1000000.0  1000000.0  1000000.0  1000000.0   
서울         95.0  1000000.0  0.0  1000000.0  1000000.0  1000000.0  1000000.0   

                     Like                                                    
                max count       mean        std  min  25%  50%   75%    max  
Location                                                                     
경기        1000000.0   4.0   1.500000   2.380476  0.0  0.0  0.5   2.0    5.0  
대전        1000000.0   1.0   0.000000        NaN  0.0  0.0  0.0   0.0    0.0  
서울        1000000.0  95.0  21.768421  38.739296  0.0  2.0  8.0  23.5  246.0

* 2개 이상의 데이터를 수집한 회사의 채용 공고의 좋아요 수 평균을 구합니다.

In [9]:
filter_df = df_sub.groupby('Company').filter(lambda x: len(x) > 1)
filter_df.groupby('Company').Like.mean()

Company
강남엄마                  13.500000
니도컴퍼니                  6.500000
닥프렌즈                  42.000000
당근마켓                   9.600000
딜리셔스(신상마켓)             5.000000
브랜디(Brandi)            5.750000
브로드씨엔에스                1.000000
소프트런치                 10.000000
스페이스워크                53.625000
아이네블루메(채티)             3.666667
에이비일팔공(AB180)         45.000000
에이치엠인터내셔날             28.500000
엘리스                   12.500000
엠포스                    2.333333
오더스팟                   4.000000
이디엠에듀케이션              19.500000
팜스킨                   43.000000
펀엔씨(강아지대통령,고양이대통령)     5.666667
Name: Like, dtype: float64